In [1]:
import sys

sys.path.append('/home/ajliang/search/search')

In [2]:
import argparse
import torch

from utils.common_utils import load_model
from map2map.map2map.data.fields import FieldDataset

In [8]:
"""Load Dataset"""
device = torch.device("cpu")

args = argparse.Namespace(
    init_input_path="/user_data/ajliang/Linear/val/LH0045/4/dis.npy",
    style_path="/user_data/ajliang/Linear/val/LH0045/4/params.npy",
    target_output_path="/user_data/ajliang/Nonlinear/val/LH0045/4/dis.npy",
    load_backward_model_state="/home/ajliang/search/model_weights/backward_model.pt",
    verbose=False,
    dropout_prob=0.0,
)

forward_dataset = FieldDataset(
    style_pattern=args.style_path,
    in_patterns=[args.init_input_path],
    tgt_patterns=[args.target_output_path],
    in_norms=['cosmology.dis'],
    tgt_norms=None,
    callback_at='/home/ajliang/search/search/map2map',
    augment=True,
    aug_shift=16,
    aug_add=None,
    aug_mul=None,
    crop=40,
    crop_start=None,
    crop_stop=None,
    crop_step=40,
    in_pad=48,
    tgt_pad=0,
    scale_factor=1
)

args.style_size = forward_dataset.style_size
args.in_chan = forward_dataset.in_chan
args.out_chan = forward_dataset.tgt_chan
backward_model = load_model(args, device=device, path_to_model_state=args.load_backward_model_state)

# style: torch.Tensor = forward_dataset[len(forward_dataset) // 2]['style']
# style = style.unsqueeze(0) # add batch dimension

forward_target: torch.Tensor = forward_dataset[len(forward_dataset) // 2]['target']
forward_target = forward_target.unsqueeze(0) # add batch dimension

backward_dataset = FieldDataset(
    style_pattern=args.style_path,
    in_patterns=[args.target_output_path],
    tgt_patterns=[args.init_input_path],
    in_norms=['cosmology.dis'],
    tgt_norms=None,
    callback_at='/home/ajliang/search/search/map2map',
    augment=True,
    aug_shift=16,
    aug_add=None,
    aug_mul=None,
    crop=40,
    crop_start=None,
    crop_stop=None,
    crop_step=40,
    in_pad=48,
    tgt_pad=0,
    scale_factor=1
)

backward_target = backward_dataset[len(backward_dataset) // 2]['input']
# add batch dimension
backward_target = backward_target.unsqueeze(0)
style = backward_dataset[len(backward_dataset) // 2]['style']

Loaded model from /home/ajliang/search/model_weights/backward_model.pt, which was trained for 128 epochs.


In [9]:
with torch.no_grad():
    init_input = backward_model(backward_target, style)

In [10]:
init_input

tensor([[[[[-7.9128e-01, -4.3660e-01, -2.4280e-02,  ..., -1.0026e+00,
            -8.4105e-01, -6.4334e-01],
           [-7.0180e-01, -3.9467e-01, -5.3336e-02,  ..., -1.1552e+00,
            -7.8313e-01, -5.7744e-01],
           [-7.1527e-01, -5.6878e-01, -2.4257e-01,  ..., -1.1448e+00,
            -7.5055e-01, -6.9462e-01],
           ...,
           [ 3.9452e-01,  4.7716e-01,  4.5620e-01,  ..., -9.0856e-01,
            -1.1139e+00, -9.2768e-01],
           [ 8.2244e-02,  2.9419e-01,  3.6302e-01,  ..., -8.3037e-01,
            -8.4866e-01, -7.5762e-01],
           [ 6.7014e-02,  1.5664e-01,  1.8783e-01,  ..., -9.7690e-01,
            -9.2157e-01, -7.7015e-01]],

          [[-3.7668e-01, -1.7484e-01, -7.2569e-02,  ..., -4.6536e-01,
            -2.3622e-01, -1.4246e-01],
           [-3.9614e-01, -3.0241e-01, -2.2620e-01,  ..., -3.3053e-01,
            -1.5471e-01, -2.0519e-01],
           [-2.0440e-01, -2.1050e-01, -3.1533e-01,  ..., -4.4198e-01,
            -2.1157e-01, -3.8453e-01],
 

In [13]:
with torch.no_grad():
    init_input = backward_model(forward_target, style)

RuntimeError: Calculated padded input size per channel: (1 x 1 x 1). Kernel size: (3 x 3 x 3). Kernel size can't be greater than actual input size

In [19]:
input = forward_dataset[0]['input']
input = input.unsqueeze(0) # add batch dimension
input.shape

torch.Size([1, 3, 136, 136, 136])

In [21]:
forward_target.shape

torch.Size([1, 3, 40, 40, 40])

In [23]:
backward_model(input, style)

tensor([[[[[ 2.2292e+00,  2.0011e+00,  1.9763e+00,  ...,  1.4610e+00,
             1.3577e+00,  1.0421e+00],
           [ 2.0884e+00,  1.9644e+00,  1.9030e+00,  ...,  1.3263e+00,
             1.0817e+00,  7.6916e-01],
           [ 2.0914e+00,  1.8502e+00,  1.7539e+00,  ...,  1.1005e+00,
             1.0406e+00,  7.4730e-01],
           ...,
           [ 1.5845e+00,  1.7285e+00,  1.6829e+00,  ...,  1.1327e+00,
             1.3160e+00,  1.3129e+00],
           [ 1.5424e+00,  1.6494e+00,  1.5292e+00,  ...,  1.1559e+00,
             1.4205e+00,  1.4900e+00],
           [ 1.5282e+00,  1.4484e+00,  1.3947e+00,  ...,  1.0037e+00,
             1.1947e+00,  1.3392e+00]],

          [[ 2.6414e+00,  2.5797e+00,  2.0901e+00,  ...,  1.3650e+00,
             1.2888e+00,  1.1327e+00],
           [ 2.7372e+00,  2.5527e+00,  2.2784e+00,  ...,  9.8262e-01,
             8.7082e-01,  8.4504e-01],
           [ 2.4347e+00,  2.4405e+00,  2.3929e+00,  ...,  1.1357e+00,
             9.6136e-01,  8.9343e-01],
 

In [ ]:
from map2map.map2map.models import StyledVNet
from map2map.map2map.utils import load_model_state_dict

model: StyledVNet = StyledVNet(
    style_size=args.style_size,
    in_chan=sum(args.in_chan),
    out_chan=sum(args.out_chan),
    dropout_prob=args.dropout_prob,
)
model.to(device)

state = torch.load("../model_weights/backward_model.pt", map_location=device)
load_model_state_dict(model, state["model"], strict=True)

In [27]:
backward_model = model
backward_model(forward_target, style)

RuntimeError: Calculated padded input size per channel: (1 x 1 x 1). Kernel size: (3 x 3 x 3). Kernel size can't be greater than actual input size

In [17]:
from map2map.map2map.data.fields import FieldDataset

dataset = FieldDataset(
    style_pattern=args.style_path,
    in_patterns=[args.init_input_path],
    tgt_patterns=[args.target_output_path],
    in_norms=['cosmology.dis'],
    tgt_norms=None,
    callback_at='/home/ajliang/search/search/map2map',
    augment=True,
    aug_shift=16,
    aug_add=None,
    aug_mul=None,
    crop=40,
    crop_start=None,
    crop_stop=None,
    crop_step=40,
    in_pad=48,
    tgt_pad=0,
    scale_factor=1
)

In [22]:
backward_model = model
backward_model()